In [1]:
%cd /content
!git clone https://github.com/Amika1118/DSGP_Group_38.git
%cd DSGP_Group_38

/content
Cloning into 'DSGP_Group_38'...
remote: Enumerating objects: 785, done.
remote: Counting objects: 100% (242/242), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 785 (delta 166), reused 44 (delta 44), pack-reused 543 (from 1)
Receiving objects: 100% (785/785), 44.72 MiB | 7.88 MiB/s, done.
Resolving deltas: 100% (302/302), done.
/content/DSGP_Group_38


In [2]:
!git checkout Market-Price-Prediction

Branch 'Market-Price-Prediction' set up to track remote branch 'Market-Price-Prediction' from 'origin'.
Switched to a new branch 'Market-Price-Prediction'


In [3]:
!git config --global user.name "Lasani Layathma"
!git config --global user.email "lasani.20241357@iit.ac.lk"

In [4]:
from getpass import getpass
token = getpass("Enter GitHub token: ")
!git remote set-url origin https://{token}@github.com/Amika1118/DSGP_Group_38.git

Enter GitHub token: ··········


In [19]:
!git pull origin Market-Price-Prediction --allow-unrelated-histories

remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 37 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Unpacking objects: 100% (37/37), 14.04 KiB | 958.00 KiB/s, done.
From https://github.com/Amika1118/DSGP_Group_38
 * branch            Market-Price-Prediction -> FETCH_HEAD
   a379bbe..4f36774  Market-Price-Prediction -> origin/Market-Price-Prediction
Updating a379bbe..4f36774
Fast-forward
 Market-Price-Prediction => Data | 0
 Images                          | 1 +
 Models                          | 1 +
 Notebooks                       | 1 +
 Outputs                         | 1 +
 5 files changed, 4 insertions(+)
 rename Market-Price-Prediction => Data (100%)
 create mode 100644 Images
 create mode 100644 Models
 create mode 100644 Notebooks
 create mode 100644 Outputs


In [5]:
# Install required libraries
!pip install pandas openpyxl

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import calendar

# Upload the file directly to Colab
from google.colab import files
print("Please upload your 'Weekly raw.xlsx' file:")
uploaded = files.upload()

# Get the filename
filename = list(uploaded.keys())[0]
print(f"\n File '{filename}' uploaded successfully!")

# Load the Retail sheet
# header=2 because the actual column headers are in row 3 (0-indexed)
df = pd.read_excel(filename, sheet_name='Retail', header=2)

print(f"\n Retail sheet loaded successfully!")
print(f"Initial shape: {df.shape}")

# Clean and prepare the dataframe
# Drop empty rows/columns
df = df.dropna(how='all').dropna(axis=1, how='all')

# Rename columns to handle them properly
# First column should be Year, second Location, third Item, then W1, W2, etc.
column_names = ['Year', 'Location', 'Item'] + [f'W{i}' for i in range(1, 53)]  # W1 to W52
# Use only the columns we need
df.columns = column_names[:len(df.columns)]

print(f"After renaming columns: {df.shape}")

# Filter for Colombo only (your requirement)
df = df[df['Location'] == 'Colombo'].copy()
print(f"\n Filtered for Colombo: {df.shape}")

# List of vegetables to include
vegetables = ['Carrot', 'Cabbage', 'Tomatoes', 'Brinjals', 'Pumpkin', 'Bitter Gourd']
df = df[df['Item'].isin(vegetables)].copy()
print(f" Filtered for 6 vegetables: {df.shape}")
print(f"Vegetables: {df['Item'].unique()}")

# Melt the dataframe - convert wide format to long format
print("\n Converting from wide format to long format...")

# Get all week columns
week_columns = [col for col in df.columns if col.startswith('W')]

# Melt the dataframe
df_melted = pd.melt(
    df,
    id_vars=['Year', 'Item'],
    value_vars=week_columns,
    var_name='Week',
    value_name='Price'
)

print(f"Melted shape: {df_melted.shape}")

# Remove rows with missing prices
df_melted = df_melted.dropna(subset=['Price']).copy()
print(f"After removing NaN prices: {df_melted.shape}")

# Extract week number from 'W1', 'W2', etc.
df_melted['Week_Num'] = df_melted['Week'].str.extract('(\d+)').astype(int)

# Convert Year and Week to actual Date
def get_date_from_year_week(year, week_num):
    """
    Convert year and week number to actual date (using Monday of that week)
    ISO week date system
    """
    try:
        # Create date for Jan 4 (always in week 1)
        jan4 = datetime(int(year), 1, 4)
        # Find Monday of week 1
        week1_monday = jan4 - timedelta(days=jan4.weekday())
        # Add weeks
        target_date = week1_monday + timedelta(weeks=(week_num - 1))
        return target_date.strftime('%Y-%m-%d')
    except:
        return None

print("\n Converting Year-Week to actual dates...")
df_melted['Date'] = df_melted.apply(
    lambda row: get_date_from_year_week(row['Year'], row['Week_Num']),
    axis=1
)

# Remove rows where date conversion failed
df_melted = df_melted.dropna(subset=['Date']).copy()
print(f"After date conversion: {df_melted.shape}")

# Create the final dataframe with just 3 columns
df_final = df_melted[['Date', 'Item', 'Price']].copy()
df_final.columns = ['Date', 'Vegetable', 'Price']

# Clean the Price column
# Convert to numeric, force errors to NaN
df_final['Price'] = pd.to_numeric(df_final['Price'], errors='coerce')
df_final = df_final.dropna(subset=['Price']).copy()

# Remove negative prices
df_final = df_final[df_final['Price'] > 0].copy()

# Sort the data
df_final = df_final.sort_values(['Date', 'Vegetable']).reset_index(drop=True)

print(f"\n Final dataset shape: {df_final.shape}")
print(f"Date range: {df_final['Date'].min()} to {df_final['Date'].max()}")
print(f"Number of unique vegetables: {df_final['Vegetable'].nunique()}")
print(f"Total records: {len(df_final)}")

# Basic statistics
print("\n")
print("BASIC STATISTICS BY VEGETABLE:")
stats = df_final.groupby('Vegetable')['Price'].agg(['count', 'mean', 'std', 'min', 'max'])
print(stats.round(2))

# Check for missing data by year
print("\n")
print("DATA COUNT BY YEAR:")
df_final['Year'] = pd.to_datetime(df_final['Date']).dt.year
year_counts = df_final.groupby(['Year', 'Vegetable']).size().unstack()
print(year_counts)

# Remove the temporary Year column
df_final = df_final.drop('Year', axis=1)

# Save to CSV file
output_filename = 'colombo_retail_prices_3col.csv'
df_final.to_csv(output_filename, index=False)
print(f"\n CSV file saved as: '{output_filename}'")


<>:69: SyntaxWarning: invalid escape sequence '\d'
<>:69: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-1488627558.py:69: SyntaxWarning: invalid escape sequence '\d'
  df_melted['Week_Num'] = df_melted['Week'].str.extract('(\d+)').astype(int)


Please upload your 'Weekly raw.xlsx' file:


Saving Weekly raw.xlsx to Weekly raw.xlsx

 File 'Weekly raw.xlsx' uploaded successfully!

 Retail sheet loaded successfully!
Initial shape: (97, 55)
After renaming columns: (97, 55)

 Filtered for Colombo: (96, 55)
 Filtered for 6 vegetables: (96, 55)
Vegetables: ['Carrot' 'Cabbage' 'Tomatoes' 'Brinjals' 'Pumpkin' 'Bitter Gourd']

 Converting from wide format to long format...
Melted shape: (4992, 4)
After removing NaN prices: (4764, 4)

 Converting Year-Week to actual dates...
After date conversion: (4764, 6)

 Final dataset shape: (4764, 3)
Date range: 2010-01-04 to 2025-12-22
Number of unique vegetables: 6
Total records: 4764


BASIC STATISTICS BY VEGETABLE:
              count    mean     std    min      max
Vegetable                                          
Bitter Gourd    794  263.67  162.54  86.60   977.78
Brinjals        794  204.00  142.30  53.38   825.17
Cabbage         794  183.06  119.31  56.78   854.33
Carrot          794  241.04  165.96  72.89  1703.33
Pumpkin         7

In [6]:
# Install and import required libraries
!pip install pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import files

# Upload your CSV file
print("Please upload your 'weekly structured.csv' file:")
uploaded = files.upload()

# Get filename
filename = list(uploaded.keys())[0]
print(f"\n File '{filename}' uploaded successfully!")

# Load the CSV
df = pd.read_csv(filename)
df['Date'] = pd.to_datetime(df['Date'])

print(f"\n Original dataset shape: {df.shape}")
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Vegetables: {df['Vegetable'].unique()}")
print("\nFirst 5 rows:")
print(df.head())

# Sort data chronologically
df = df.sort_values(['Vegetable', 'Date']).reset_index(drop=True)
print("\n Data sorted by Vegetable and Date")

# Create a function to add all features for each vegetable
def add_all_features(df):
    """Add all preprocessing columns to the dataframe"""

    # Make a copy to avoid warnings
    df = df.copy()

    # 1. TIME FEATURES (From Date column)
    print("\n Adding TIME FEATURES...")
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Week_of_Year'] = df['Date'].dt.isocalendar().week
    df['Quarter'] = df['Date'].dt.quarter

    # 2. ONE-HOT ENCODING (Vegetable indicators)
    print(" Adding ONE-HOT ENCODING...")
    vegetables = ['Bitter Gourd', 'Brinjals', 'Cabbage', 'Carrot', 'Pumpkin', 'Tomatoes']
    for veg in vegetables:
        col_name = f'Is_{veg.replace(" ", "_")}'
        df[col_name] = (df['Vegetable'] == veg).astype(int)

    # 3. LAG FEATURES (Past prices)
    print(" Adding LAG FEATURES...")

    # Sort to ensure correct order for lag calculation
    df = df.sort_values(['Vegetable', 'Date']).reset_index(drop=True)

    # Group by vegetable to calculate lags correctly
    for veg in df['Vegetable'].unique():
        mask = df['Vegetable'] == veg

        # Price lags
        df.loc[mask, 'Price_Lag_1'] = df.loc[mask, 'Price'].shift(1)
        df.loc[mask, 'Price_Lag_2'] = df.loc[mask, 'Price'].shift(2)
        df.loc[mask, 'Price_Lag_3'] = df.loc[mask, 'Price'].shift(3)
        df.loc[mask, 'Price_Lag_4'] = df.loc[mask, 'Price'].shift(4)
        df.loc[mask, 'Price_Lag_8'] = df.loc[mask, 'Price'].shift(8)
        df.loc[mask, 'Price_Lag_12'] = df.loc[mask, 'Price'].shift(12)
        df.loc[mask, 'Price_Lag_52'] = df.loc[mask, 'Price'].shift(52)

    # 4. ROLLING STATISTICS (Trends over time)
    print(" Adding ROLLING STATISTICS...")

    for veg in df['Vegetable'].unique():
        mask = df['Vegetable'] == veg

        # Rolling means
        df.loc[mask, 'Rolling_Mean_4'] = df.loc[mask, 'Price'].rolling(window=4, min_periods=1).mean()
        df.loc[mask, 'Rolling_Mean_8'] = df.loc[mask, 'Price'].rolling(window=8, min_periods=1).mean()
        df.loc[mask, 'Rolling_Mean_12'] = df.loc[mask, 'Price'].rolling(window=12, min_periods=1).mean()

        # Rolling min and max
        df.loc[mask, 'Rolling_Min_4'] = df.loc[mask, 'Price'].rolling(window=4, min_periods=1).min()
        df.loc[mask, 'Rolling_Max_4'] = df.loc[mask, 'Price'].rolling(window=4, min_periods=1).max()

        # Rolling standard deviation (volatility)
        df.loc[mask, 'Rolling_Std_4'] = df.loc[mask, 'Price'].rolling(window=4, min_periods=1).std()

    # 5. PRICE CHANGE FEATURES
    print(" Adding PRICE CHANGE FEATURES...")

    for veg in df['Vegetable'].unique():
        mask = df['Vegetable'] == veg

        # Price change from 1 week ago
        df.loc[mask, 'Price_Change_1wk'] = df.loc[mask, 'Price'] - df.loc[mask, 'Price'].shift(1)

        # Percentage change from 1 week ago
        df.loc[mask, 'Price_Change_Pct_1wk'] = (df.loc[mask, 'Price'] - df.loc[mask, 'Price'].shift(1)) / df.loc[mask, 'Price'].shift(1) * 100

        # Price change from 4 weeks ago
        df.loc[mask, 'Price_Change_4wk'] = df.loc[mask, 'Price'] - df.loc[mask, 'Price'].shift(4)

    # 6. SEASONAL FEATURES (Monthly averages)
    print(" Adding SEASONAL FEATURES...")

    # Calculate monthly average price for each vegetable
    monthly_avg = df.groupby(['Vegetable', 'Month'])['Price'].transform('mean')
    df['Month_Avg_Price'] = monthly_avg

    # Compare current price to monthly average
    df['Price_vs_Month_Avg'] = df['Price'] - df['Month_Avg_Price']

    # 7. CLEAN UP - Remove rows with NaN values (first few weeks)
    print(" Cleaning up NaN values...")

    # For time series, we need at least Price_Lag_1 to make predictions
    initial_rows = len(df)
    df = df.dropna(subset=['Price_Lag_1']).reset_index(drop=True)
    removed_rows = initial_rows - len(df)
    print(f"   Removed {removed_rows} rows with NaN lag values (first weeks of data)")

    # Fill any remaining NaN values in rolling stats with forward fill
    df = df.fillna(method='ffill')

    # Fill any remaining NaN values with 0
    df = df.fillna(0)

    return df

# Apply the feature engineering function
print("\n")
print(" STARTING FEATURE ENGINEERING...")

df_featured = add_all_features(df)

print("\n Feature engineering complete!")
print(f"   New dataset shape: {df_featured.shape}")
print(f"   Total columns: {len(df_featured.columns)}")

# Reorder columns to match your requested format
print("\n Reordering columns...")

# Define the exact column order you requested
column_order = [
    'Date',
    'Vegetable',
    'Price',  # TARGET
    'Year',
    'Month',
    'Week_of_Year',
    'Quarter',
    'Price_Lag_1',
    'Price_Lag_2',
    'Price_Lag_3',
    'Price_Lag_4',
    'Price_Lag_8',
    'Price_Lag_12',
    'Price_Lag_52',
    'Rolling_Mean_4',
    'Rolling_Mean_8',
    'Rolling_Mean_12',
    'Rolling_Std_4',
    'Rolling_Min_4',
    'Rolling_Max_4',
    'Price_Change_1wk',
    'Price_Change_Pct_1wk',
    'Price_Change_4wk',
    'Month_Avg_Price',
    'Price_vs_Month_Avg',
    'Is_Bitter_Gourd',
    'Is_Brinjals',
    'Is_Cabbage',
    'Is_Carrot',
    'Is_Pumpkin',
    'Is_Tomatoes'
]

# Add only columns that exist in our dataframe
final_columns = [col for col in column_order if col in df_featured.columns]
df_final = df_featured[final_columns].copy()

print(f" Final dataset has {len(final_columns)} columns")

# Display column information
print("\n")
print(" COLUMN INFORMATION")
for i, col in enumerate(df_final.columns, 1):
    dtype = df_final[col].dtype
    non_null = df_final[col].count()
    print(f"{i:2d}. {col:25s} | dtype: {str(dtype):8s} | non-null: {non_null:6d}")

# Basic statistics of the preprocessed data
print("\n")
print(" BASIC STATISTICS (Numerical Columns)")
print(df_final.describe().round(2))

# Check for any remaining missing values
print("\n")
print(" MISSING VALUES CHECK")
missing_values = df_final.isnull().sum()
if missing_values.sum() == 0:
    print(" No missing values found! Dataset is clean.")
else:
    print(missing_values[missing_values > 0])

# Verify lag features are working correctly
print("\n")
print(" VERIFYING LAG FEATURES (Carrot example)")
carrot_sample = df_final[df_final['Is_Carrot'] == 1].head(10)
print(carrot_sample[['Date', 'Price', 'Price_Lag_1', 'Price_Lag_2', 'Price_Change_1wk']].to_string())

# Save the preprocessed dataset
output_filename = 'weekly_preprocessed_complete.csv'
df_final.to_csv(output_filename, index=False)
print(f"\n Preprocessed dataset saved as: '{output_filename}'")
print(f"   File size: {len(df_final)} rows × {len(df_final.columns)} columns")
'''
 FEATURE SUMMARY:
   • Time Features: Year, Month, Week_of_Year, Quarter
   • Lag Features: Price_Lag_1,2,3,4,8,12,52
   • Rolling Stats: Mean_4,8,12, Std_4, Min_4, Max_4
   • Price Changes: 1wk, 4wk, Pct_1wk
   • Seasonal: Month_Avg_Price, Price_vs_Month_Avg
   • Encoding: 6 one-hot columns for vegetables'''


Please upload your 'weekly structured.csv' file:


Saving weekly structured.csv to weekly structured.csv

 File 'weekly structured.csv' uploaded successfully!

 Original dataset shape: (4764, 3)
Date range: 2010-01-04 00:00:00 to 2025-12-22 00:00:00
Vegetables: ['Bitter Gourd' 'Brinjals' 'Cabbage' 'Carrot' 'Pumpkin' 'Tomatoes']

First 5 rows:
        Date     Vegetable   Price
0 2010-01-04  Bitter Gourd  114.12
1 2010-01-04      Brinjals  113.17
2 2010-01-04       Cabbage   93.78
3 2010-01-04        Carrot  129.76
4 2010-01-04       Pumpkin   60.00

 Data sorted by Vegetable and Date


 STARTING FEATURE ENGINEERING...

 Adding TIME FEATURES...
 Adding ONE-HOT ENCODING...
 Adding LAG FEATURES...
 Adding ROLLING STATISTICS...
 Adding PRICE CHANGE FEATURES...
 Adding SEASONAL FEATURES...
 Cleaning up NaN values...
   Removed 6 rows with NaN lag values (first weeks of data)

 Feature engineering complete!
   New dataset shape: (4758, 31)
   Total columns: 31

 Reordering columns...
 Final dataset has 31 columns


 COLUMN INFORMATION
 1. Da

/tmp/ipython-input-1352587133.py:124: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


                                Date    Price     Year    Month  Week_of_Year  \
count                           4758  4758.00  4758.00  4758.00        4758.0   
mean   2017-11-21 14:08:01.210592512   205.94  2017.39     6.51         26.51   
min              2010-01-11 00:00:00    44.19  2010.00     1.00           1.0   
25%              2013-11-11 00:00:00   106.36  2013.00     4.00          13.0   
50%              2017-09-04 00:00:00   157.03  2017.00     7.00          27.0   
75%              2022-03-07 00:00:00   255.71  2022.00     9.00          39.0   
max              2025-12-22 00:00:00  1703.33  2025.00    12.00          52.0   
std                              NaN   147.41     4.67     3.44         15.01   

       Quarter  Price_Lag_1  Price_Lag_2  Price_Lag_3  Price_Lag_4  ...  \
count  4758.00      4758.00      4758.00      4758.00      4758.00  ...   
mean      2.51       205.35       205.17       205.00       204.95  ...   
min       1.00        44.19         0.00     

'\n FEATURE SUMMARY:\n   • Time Features: Year, Month, Week_of_Year, Quarter\n   • Lag Features: Price_Lag_1,2,3,4,8,12,52\n   • Rolling Stats: Mean_4,8,12, Std_4, Min_4, Max_4\n   • Price Changes: 1wk, 4wk, Pct_1wk\n   • Seasonal: Month_Avg_Price, Price_vs_Month_Avg\n   • Encoding: 6 one-hot columns for vegetables'

In [7]:
# Install and import required libraries
!pip install pandas numpy

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from google.colab import files

# Upload fuel prices CSV file
print("Please upload your 'fuel_prices (1).csv' file:")
uploaded = files.upload()

# Get filename
filename = list(uploaded.keys())[0]
print(f"\n File '{filename}' uploaded successfully!")

# Load the CSV
df_fuel = pd.read_csv(filename)

print(f"\n Original dataset shape: {df_fuel.shape}")
print(f"Date range in original: {pd.to_datetime(df_fuel['date']).min()} to {pd.to_datetime(df_fuel['date']).max()}")

# Clean and prepare the fuel data
print("\n")
print(" PROCESSING FUEL PRICE DATA (2010-2025 ONLY)...")

# Make a copy to work with
df = df_fuel.copy()

# Convert date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Keep only relevant columns and rename
df = df[['date', 'auto_diesel']].copy()
df.columns = ['Date', 'Fuel_Price']

# Sort by date
df = df.sort_values('Date').reset_index(drop=True)

# FILTER FOR 2010-2025 ONLY
start_date = '2010-01-01'
end_date = '2025-12-31'

df_filtered = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)].copy()

print(f"\n Filtered to 2010-2025:")
print(f"   Original rows: {len(df)}")
print(f"   Filtered rows: {len(df_filtered)}")
print(f"   Date range: {df_filtered['Date'].min()} to {df_filtered['Date'].max()}")
print(f"   Fuel price range: Rs.{df_filtered['Fuel_Price'].min()} to Rs.{df_filtered['Fuel_Price'].max()}")

# Create complete weekly date range for 2010-2025
print("\n Creating weekly date range (2010-2025)...")

# Create weekly dates from 2010 to 2025 (every Monday)
weekly_dates = pd.date_range(start='2010-01-04', end='2025-12-29', freq='W-MON')
weekly_df = pd.DataFrame({'Date': weekly_dates})

print(f"   Created {len(weekly_df)} weekly dates from {weekly_df['Date'].min()} to {weekly_df['Date'].max()}")

# Merge and interpolate fuel prices to weekly frequency
print("\n Interpolating fuel prices to weekly frequency...")

# Merge weekly dates with actual fuel price changes
df_weekly = pd.merge(weekly_df, df_filtered, on='Date', how='left')

# Forward fill to get the most recent fuel price for each week
# (Fuel prices remain constant until the next price change)
df_weekly['Fuel_Price'] = df_weekly['Fuel_Price'].fillna(method='ffill')

# Also backward fill for any missing at the beginning
df_weekly['Fuel_Price'] = df_weekly['Fuel_Price'].fillna(method='bfill')

print(f" Weekly fuel data: {len(df_weekly)} rows")
print(f"   Missing values after interpolation: {df_weekly['Fuel_Price'].isna().sum()}")

# Format Date as YYYY-MM-DD string
df_weekly['Date'] = df_weekly['Date'].dt.strftime('%Y-%m-%d')

# Display statistics for 2010-2025 period
print("\n")
print(" FUEL PRICE STATISTICS (2010-2025)")
print(f"Total weeks: {len(df_weekly)}")
print(f"Date range: {df_weekly['Date'].min()} to {df_weekly['Date'].max()}")
print(f"Min price: Rs.{df_weekly['Fuel_Price'].min():.2f}")
print(f"Max price: Rs.{df_weekly['Fuel_Price'].max():.2f}")
print(f"Mean price: Rs.{df_weekly['Fuel_Price'].mean():.2f}")
print(f"Median price: Rs.{df_weekly['Fuel_Price'].median():.2f}")
print(f"Std deviation: Rs.{df_weekly['Fuel_Price'].std():.2f}")

# Count price changes in 2010-2025
price_changes = df_weekly['Fuel_Price'].diff().fillna(0) != 0
n_changes = price_changes.sum()
print(f"\nNumber of fuel price changes (2010-2025): {n_changes}")
print(f"Average days between changes: {len(df_weekly) / n_changes * 7:.1f} days")

# Show fuel price at start and end of period
print(f"\n Fuel price on {df_weekly['Date'].iloc[0]}: Rs.{df_weekly['Fuel_Price'].iloc[0]:.2f}")
print(f" Fuel price on {df_weekly['Date'].iloc[-1]}: Rs.{df_weekly['Fuel_Price'].iloc[-1]:.2f}")
print(f" Total increase: Rs.{df_weekly['Fuel_Price'].iloc[-1] - df_weekly['Fuel_Price'].iloc[0]:.2f}")

# Save the processed fuel price data (2010-2025 only)
output_filename = 'weekly_fuel_prices_2010_2025.csv'
df_weekly.to_csv(output_filename, index=False)
print(f"\n Weekly fuel prices (2010-2025) saved as: '{output_filename}'")
print(f"   File: {len(df_weekly)} rows × 2 columns")

# Also save the original filtered price changes (for reference)
print("\n Saving original fuel price change dates (2010-2025)...")
df_original_filtered = df_filtered.copy()
df_original_filtered['Date'] = df_original_filtered['Date'].dt.strftime('%Y-%m-%d')
original_filename = 'fuel_price_changes_2010_2025.csv'
df_original_filtered.to_csv(original_filename, index=False)
print(f" Original price changes (2010-2025) saved as: '{original_filename}'")
print(f"   Records: {len(df_original_filtered)} price changes")
files.download(original_filename)

Please upload your 'fuel_prices (1).csv' file:


Saving fuel_prices (1).csv to fuel_prices (1).csv

 File 'fuel_prices (1).csv' uploaded successfully!

 Original dataset shape: (170, 16)
Date range in original: 1990-03-01 00:00:00 to 2025-11-01 00:00:00


 PROCESSING FUEL PRICE DATA (2010-2025 ONLY)...

 Filtered to 2010-2025:
   Original rows: 170
   Filtered rows: 91
   Date range: 2010-09-01 00:00:00 to 2025-11-01 00:00:00
   Fuel price range: Rs.73.0 to Rs.460.0

 Creating weekly date range (2010-2025)...
   Created 835 weekly dates from 2010-01-04 00:00:00 to 2025-12-29 00:00:00

 Interpolating fuel prices to weekly frequency...
 Weekly fuel data: 835 rows
   Missing values after interpolation: 0


 FUEL PRICE STATISTICS (2010-2025)
Total weeks: 835
Date range: 2010-01-04 to 2025-12-29
Min price: Rs.121.00
Max price: Rs.430.00
Mean price: Rs.170.61
Median price: Rs.121.00
Std deviation: Rs.93.27

Number of fuel price changes (2010-2025): 11
Average days between changes: 531.4 days

 Fuel price on 2010-01-04: Rs.121.00
 Fuel pric

/tmp/ipython-input-2442955923.py:69: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_weekly['Fuel_Price'] = df_weekly['Fuel_Price'].fillna(method='ffill')
/tmp/ipython-input-2442955923.py:72: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_weekly['Fuel_Price'] = df_weekly['Fuel_Price'].fillna(method='bfill')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
import pandas as pd

# Read both CSV files
fuel_df = pd.read_csv('weekly_fuel_prices_2010_2025.csv')
veg_df = pd.read_csv('weekly_preprocessed_complete.csv')

# Ensure date columns are in datetime format for proper merging
fuel_df['Date'] = pd.to_datetime(fuel_df['Date'])
veg_df['Date'] = pd.to_datetime(veg_df['Date'])

# Perform left merge to keep all vegetable data and add fuel prices
merged_df = pd.merge(veg_df, fuel_df[['Date', 'Fuel_Price']], on='Date', how='left')

# Save the merged dataframe to a new CSV file
merged_df.to_csv('weekly_preprocessed_with_fuel.csv', index=False)

print(f"Original vegetable dataset rows: {len(veg_df)}")
print(f"Original fuel dataset rows: {len(fuel_df)}")
print(f"Merged dataset rows: {len(merged_df)}")
print(f"\nColumns in merged dataset: {merged_df.columns.tolist()}")
print(f"\nFuel price statistics:")
print(merged_df['Fuel_Price'].describe())
print(f"\nMissing fuel prices: {merged_df['Fuel_Price'].isna().sum()}")

# Display first few rows to verify the merge
print("\nFirst 5 rows of merged dataset:")
print(merged_df[['Date', 'Vegetable', 'Price', 'Fuel_Price']].head())

Original vegetable dataset rows: 4758
Original fuel dataset rows: 835
Merged dataset rows: 4758

Columns in merged dataset: ['Date', 'Vegetable', 'Price', 'Year', 'Month', 'Week_of_Year', 'Quarter', 'Price_Lag_1', 'Price_Lag_2', 'Price_Lag_3', 'Price_Lag_4', 'Price_Lag_8', 'Price_Lag_12', 'Price_Lag_52', 'Rolling_Mean_4', 'Rolling_Mean_8', 'Rolling_Mean_12', 'Rolling_Std_4', 'Rolling_Min_4', 'Rolling_Max_4', 'Price_Change_1wk', 'Price_Change_Pct_1wk', 'Price_Change_4wk', 'Month_Avg_Price', 'Price_vs_Month_Avg', 'Is_Bitter_Gourd', 'Is_Brinjals', 'Is_Cabbage', 'Is_Carrot', 'Is_Pumpkin', 'Is_Tomatoes', 'Fuel_Price']

Fuel price statistics:
count    4758.000000
mean      173.036570
std        94.879005
min       121.000000
25%       121.000000
50%       121.000000
75%       121.000000
max       430.000000
Name: Fuel_Price, dtype: float64

Missing fuel prices: 0

First 5 rows of merged dataset:
        Date     Vegetable   Price  Fuel_Price
0 2010-01-11  Bitter Gourd  116.13       121.0
1 2